IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
import cv2
import mediapipe as mp
import math
import keyboard # Acho que nao vou usar
import pandas as pd # Tabelas
from datetime import datetime 
import serial
import time
